In [1]:
import turicreate as tc

In [2]:
# Load data 
user_tour_data = tc.SFrame.read_csv('./dataset/user_tour_data.csv', verbose=False)
user_data = tc.SFrame.read_csv('./dataset/user_data.csv', verbose=False)
tour_data = tc.SFrame.read_csv('./dataset/tour_data.csv', verbose=False)

In [3]:
user_tour_data.print_rows(user_tour_data.shape[0]) #What users went on what tours

+--------+--------+
| userId | tourId |
+--------+--------+
|   1    |   1    |
|   2    |   3    |
|   3    |   1    |
|   3    |   2    |
|   1    |   3    |
|   4    |   2    |
|   4    |   4    |
|   5    |   2    |
|   6    |   3    |
|   7    |   3    |
|   6    |   2    |
+--------+--------+
[11 rows x 2 columns]



In [4]:
user_data.print_rows(user_data.shape[0]) #Info about the users

+--------+-----+--------+
| userId | age | gender |
+--------+-----+--------+
|   1    |  21 |   M    |
|   2    |  80 |   M    |
|   3    |  35 |   F    |
|   4    |  35 |   M    |
|   5    |  22 |   F    |
|   6    |  60 |   F    |
|   7    |  70 |   M    |
+--------+-----+--------+
[7 rows x 3 columns]



In [5]:
tour_data.print_rows(tour_data.shape[0]) #Info about the tours

+--------+------------+------------+
| tourId | lengthDist | lengthTime |
+--------+------------+------------+
|   1    |     5      |     20     |
|   2    |     15     |     45     |
|   3    |     10     |     60     |
|   4    |     25     |     90     |
+--------+------------+------------+
[4 rows x 3 columns]



In [6]:
# Train-test split
training_data, validation_data = tc.recommender.util.random_split_by_user(user_tour_data, 'userId', 'tourId')

In [7]:
#Create model

model = tc.recommender.ranking_factorization_recommender.create(training_data, 'userId', 'tourId',
                                                                          user_data=user_data,
                                                                          item_data=tour_data)
#Model choice: Ranking Factorization Recommender
#Reasons: Implicit data, lack of ratings, additional relevant user & tour data
#More info here: https://turi.com/learn/userguide/recommender/choosing-a-model.html

Preparing data set.

Data has 9 observations with 7 users and 4 items.

Data prepared in: 0.012366s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------------------+-------------+

| Initial | 117us        | 1.38637           | 0.693303                          |             |

+---------+--------------+-------------------+-----------------------------------+-------------+

| 1       | 1.487ms      | DIVERGED          | DIVERGED                          | 0.130208    |

| RESET   | 2.016ms      | 1.38641           | 0.693253                          |             |

In [10]:
#Print results
results = model.recommend()
results.print_rows(results.shape[0])
#All the tours a given user has NOT been on are ranked according to which ones the user is most likely to favor

+--------+--------+-----------------------+------+
| userId | tourId |         score         | rank |
+--------+--------+-----------------------+------+
|   1    |   4    |  0.03433328687311106  |  1   |
|   1    |   2    |  0.011053869673404045 |  2   |
|   2    |   2    |  0.12445551588192133  |  1   |
|   2    |   1    |  0.11582134601180201  |  2   |
|   2    |   4    |  0.03454589552404447  |  3   |
|   3    |   3    |  0.02165207903766628  |  1   |
|   3    |   4    |  0.009644967743661238 |  2   |
|   4    |   3    |  0.08161424981289148  |  1   |
|   4    |   1    |  0.08126687394137724  |  2   |
|   4    |   4    | 0.0055765395746838675 |  3   |
|   5    |   3    |  0.07347743598060547  |  1   |
|   5    |   1    |   0.0658364620312423  |  2   |
|   5    |   4    | 0.0074409134382697915 |  3   |
|   6    |   1    |  0.059294657650934335 |  1   |
|   6    |   4    |  0.03672864879546999  |  2   |
|   7    |   3    |   0.8137699543872561  |  1   |
|   7    |   2    |   0.5911615

### <i>(Analysis / validation of model pending.)</i>